In [1]:
import os
import io
import csv
from read_data import convert_pdf_to_txt
from PIL import Image
import pytesseract
from wand.image import Image as wi
import hashlib
import pandas as pd
import numpy as np
import pprint as pp
from urllib.request import urlopen
import time

In [45]:
def conv_pdf(pdf):
    if os.path.isfile('./pdfs/' + pdf):
        with open('./pdfs/' + pdf, 'rb') as f:
            # text from pdf
            bfr = io.BufferedReader(f)
            pdf_text = convert_pdf_to_txt(bfr) 

            # recognized text (OCR) from pdf
            if len(pdf_text.strip()) <= 50:
                with wi(filename='./pdfs/' + pdf, resolution=200) as pdf_file:
                    pdfImage = pdf_file.convert('jpeg')
                    imageBlobs = []
                    for img in pdfImage.sequence:
                        with wi(image = img) as imgPage:
                            imageBlobs.append(imgPage.make_blob('jpeg'))

                recognized_text = []

                for imgBlob in imageBlobs:
                    im = Image.open(io.BytesIO(imgBlob))
                    text = pytesseract.image_to_string(im, lang = 'eng')
                    recognized_text.append(text)

                recognized_text = '\n\n\n'.join(recognized_text)

            pdf_text = pdf_text if len(pdf_text.strip()) > 50 else recognized_text

            print('--> ' + pdf)

            return pdf_text

In [ ]:
def write_pdf(path):
    pdf_doc = urlopen(path)
    time.sleep(4)
    with open(path.split('/')[-1], 'wb') as w:
        w.write(pdf_doc.read())
        print('--> ' + path.split('/')[-1])
        return path.split('/')[-1]

In [3]:
kansas_csv = pd.read_csv('kansas_20170830.csv', index_col = 1)[1001:2001] # change the list len
kansas_csv.head()

,bill,session,text,url,filename
committee,,,,,
Committee on Corrections and Juvenile Justice,NaN,2011-2012 Legislative Sessions,NaN,http://www.kslegislature.org/li_2012/b2011_12/...,li_2012_b2011_12_committees_resources_ctte_h_c...
Committee on Corrections and Juvenile Justice,NaN,2011-2012 Legislative Sessions,NaN,http://www.kslegislature.org/li_2012/b2011_12/...,li_2012_b2011_12_committees_resources_ctte_h_c...
Committee on Corrections and Juvenile Justice,NaN,2011-2012 Legislative Sessions,NaN,http://www.kslegislature.org/li_2012/b2011_12/...,li_2012_b2011_12_committees_resources_ctte_h_c...
Committee on Corrections and Juvenile Justice,NaN,2011-2012 Legislative Sessions,NaN,http://www.kslegislature.org/li_2012/b2011_12/...,li_2012_b2011_12_committees_resources_ctte_h_c...
Committee on Corrections and Juvenile Justice,NaN,2011-2012 Legislative Sessions,NaN,http://www.kslegislature.org/li_2012/b2011_12/...,li_2012_b2011_12_committees_resources_ctte_h_c...


In [ ]:
kansas_csv['text'] = kansas_csv.filename.apply(conv_pdf)

In [4]:
kansas_csv.head()

,bill,session,text,url,filename
committee,,,,,
Committee on Corrections and Juvenile Justice,NaN,2011-2012 Legislative Sessions,NaN,http://www.kslegislature.org/li_2012/b2011_12/...,li_2012_b2011_12_committees_resources_ctte_h_c...
Committee on Corrections and Juvenile Justice,NaN,2011-2012 Legislative Sessions,NaN,http://www.kslegislature.org/li_2012/b2011_12/...,li_2012_b2011_12_committees_resources_ctte_h_c...
Committee on Corrections and Juvenile Justice,NaN,2011-2012 Legislative Sessions,NaN,http://www.kslegislature.org/li_2012/b2011_12/...,li_2012_b2011_12_committees_resources_ctte_h_c...
Committee on Corrections and Juvenile Justice,NaN,2011-2012 Legislative Sessions,NaN,http://www.kslegislature.org/li_2012/b2011_12/...,li_2012_b2011_12_committees_resources_ctte_h_c...
Committee on Corrections and Juvenile Justice,NaN,2011-2012 Legislative Sessions,NaN,http://www.kslegislature.org/li_2012/b2011_12/...,li_2012_b2011_12_committees_resources_ctte_h_c...


In [ ]:
kansas_csv.to_csv('kansas1001-1890.csv')

In [129]:
fil_csv = pd.read_csv('kansas1001-1890.csv', index_col=0)

In [130]:
b = fil_csv[fil_csv[['text']].isnull().values]

In [137]:
b.loc[:, 'filename'] = b.url.apply(write_pdf)

--> ctte_s_fin_inst_ins_1_20120327_min.pdf
--> ctte_s_fin_inst_ins_1_20120315_min.pdf
--> ctte_s_fin_inst_ins_1_20120314_min.pdf
--> ctte_s_fin_inst_ins_1_20120313_min.pdf
--> ctte_s_fin_inst_ins_1_20120308_min.pdf
--> ctte_s_fin_inst_ins_1_20120307_min.pdf
--> ctte_s_fin_inst_ins_1_20120306_min.pdf
--> ctte_s_fin_inst_ins_1_20120301_min.pdf
--> ctte_s_fin_inst_ins_1_20120221_min.pdf
--> ctte_s_fin_inst_ins_1_20120220_min.pdf
--> ctte_s_fin_inst_ins_1_20120216_min.pdf
--> ctte_s_fin_inst_ins_1_20120215_min.pdf
--> ctte_s_fin_inst_ins_1_20120214_min.pdf
--> ctte_s_fin_inst_ins_1_20120209_min.pdf
--> ctte_s_fin_inst_ins_1_20120208_min.pdf
--> ctte_s_fin_inst_ins_1_20120207_min.pdf
--> ctte_s_fin_inst_ins_1_20120202_min.pdf
--> ctte_s_fin_inst_ins_1_20120201_min.pdf
--> ctte_s_fin_inst_ins_1_20120131_min.pdf
--> ctte_s_fin_inst_ins_1_20120126_min.pdf
--> ctte_s_fin_inst_ins_1_20120125_min.pdf
--> ctte_s_fin_inst_ins_1_20120124_min.pdf
--> ctte_s_fin_inst_ins_1_20120119_min.pdf
--> ctte_s_

/Users/zackushka/anaconda/envs/scr/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [141]:
b['text'] = b.filename.apply(conv_pdf)

--> ctte_s_fin_inst_ins_1_20120327_min.pdf
--> ctte_s_fin_inst_ins_1_20120315_min.pdf
--> ctte_s_fin_inst_ins_1_20120314_min.pdf
--> ctte_s_fin_inst_ins_1_20120313_min.pdf
--> ctte_s_fin_inst_ins_1_20120308_min.pdf
--> ctte_s_fin_inst_ins_1_20120307_min.pdf
--> ctte_s_fin_inst_ins_1_20120306_min.pdf
--> ctte_s_fin_inst_ins_1_20120301_min.pdf
--> ctte_s_fin_inst_ins_1_20120221_min.pdf
--> ctte_s_fin_inst_ins_1_20120220_min.pdf
--> ctte_s_fin_inst_ins_1_20120216_min.pdf
--> ctte_s_fin_inst_ins_1_20120215_min.pdf
--> ctte_s_fin_inst_ins_1_20120214_min.pdf
--> ctte_s_fin_inst_ins_1_20120209_min.pdf
--> ctte_s_fin_inst_ins_1_20120208_min.pdf
--> ctte_s_fin_inst_ins_1_20120207_min.pdf
--> ctte_s_fin_inst_ins_1_20120202_min.pdf
--> ctte_s_fin_inst_ins_1_20120201_min.pdf
--> ctte_s_fin_inst_ins_1_20120131_min.pdf
--> ctte_s_fin_inst_ins_1_20120126_min.pdf
--> ctte_s_fin_inst_ins_1_20120125_min.pdf
--> ctte_s_fin_inst_ins_1_20120124_min.pdf
--> ctte_s_fin_inst_ins_1_20120119_min.pdf
--> ctte_s_

/Users/zackushka/anaconda/envs/scr/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [145]:
b

,bill,session,text,url,filename
committee,,,,,
Committee on Financial Institutions and Insurance,NaN,2011-2012 Legislative Sessions,\n\nApproved: \n\n \n\n4-25-2012 \n\n(Date) \...,http://www.kslegislature.org/li_2012/b2011_12/...,ctte_s_fin_inst_ins_1_20120327_min.pdf
Committee on Financial Institutions and Insurance,NaN,2011-2012 Legislative Sessions,\n\nApproved: \n\n03-30-2012 \n\n \n\n(Date) ...,http://www.kslegislature.org/li_2012/b2011_12/...,ctte_s_fin_inst_ins_1_20120315_min.pdf
Committee on Financial Institutions and Insurance,NaN,2011-2012 Legislative Sessions,\n\nApproved: \n\n03-30-2012 \n\n \n\n(Date) ...,http://www.kslegislature.org/li_2012/b2011_12/...,ctte_s_fin_inst_ins_1_20120314_min.pdf
Committee on Financial Institutions and Insurance,NaN,2011-2012 Legislative Sessions,\n\nApproved: \n\n03-30-2012 \n\n \n\n(Date) ...,http://www.kslegislature.org/li_2012/b2011_12/...,ctte_s_fin_inst_ins_1_20120313_min.pdf
Committee on Financial Institutions and Insurance,NaN,2011-2012 Legislative Sessions,\n\nApproved: \n\n03-30-2012 \n\n \n\n(Date) ...,http://www.kslegislature.org/li_2012/b2011_12/...,ctte_s_fin_inst_ins_1_20120308_min.pdf
Committee on Financial Institutions and Insurance,NaN,2011-2012 Legislative Sessions,\n\nApproved: \n\n03-30-2012 \n\n \n\n(Date) ...,http://www.kslegislature.org/li_2012/b2011_12/...,ctte_s_fin_inst_ins_1_20120307_min.pdf
Committee on Financial Institutions and Insurance,NaN,2011-2012 Legislative Sessions,3-6-12 \nChairman Teichman opened the meeting ...,http://www.kslegislature.org/li_2012/b2011_12/...,ctte_s_fin_inst_ins_1_20120306_min.pdf
Committee on Financial Institutions and Insurance,NaN,2011-2012 Legislative Sessions,\n\nApproved: \n\n03-30-2012 \n\n \n\n(Date) ...,http://www.kslegislature.org/li_2012/b2011_12/...,ctte_s_fin_inst_ins_1_20120301_min.pdf
Committee on Financial Institutions and Insurance,NaN,2011-2012 Legislative Sessions,\n\nApproved: \n\n03-26-2012 \n\n \n\n(Date) ...,http://www.kslegislature.org/li_2012/b2011_12/...,ctte_s_fin_inst_ins_1_20120221_min.pdf


In [146]:
b.to_csv('kansas_missed_115.csv')

In [111]:
b.loc[:, 'filename'] = fil_csv[fil_csv[['text']].isnull().values].url.apply(write_pdf)

--> ctte_h_fed_st_1_20120209_min.pdf
--> ctte_h_fed_st_1_20120208_min.pdf
--> ctte_h_fed_st_1_20120207_min.pdf
--> ctte_h_fed_st_1_20120206_min.pdf
--> ctte_h_fed_st_1_20120202_min.pdf
--> ctte_h_fed_st_1_20120201_min.pdf
--> ctte_h_fed_st_1_20120131_min.pdf
--> ctte_h_fed_st_1_20120130_min.pdf
--> ctte_h_fed_st_1_20120126_min.pdf
--> ctte_h_fed_st_1_20120125_min.pdf
--> ctte_h_fed_st_1_20120124_min.pdf
--> ctte_h_fed_st_1_20120123_min.pdf
--> ctte_h_fed_st_1_20120119_min.pdf
--> ctte_h_fed_st_1_20120118_min.pdf
--> ctte_h_fed_st_1_20120117_min.pdf
--> ctte_h_fed_st_1_20120111_min.pdf
--> ctte_h_fed_st_1_20120111_min.pdf
--> ctte_h_fed_st_1_20110505_min.pdf
--> ctte_h_fed_st_1_20110330_min.pdf
--> ctte_h_fed_st_1_20110322_min.pdf


/Users/zackushka/anaconda/envs/scr/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [118]:
b['text'] = b.filename.apply(conv_pdf)

--> ctte_h_fed_st_1_20120209_min.pdf
--> ctte_h_fed_st_1_20120208_min.pdf
--> ctte_h_fed_st_1_20120207_min.pdf
--> ctte_h_fed_st_1_20120206_min.pdf
--> ctte_h_fed_st_1_20120202_min.pdf
--> ctte_h_fed_st_1_20120201_min.pdf
--> ctte_h_fed_st_1_20120131_min.pdf
--> ctte_h_fed_st_1_20120130_min.pdf
--> ctte_h_fed_st_1_20120126_min.pdf
--> ctte_h_fed_st_1_20120125_min.pdf
--> ctte_h_fed_st_1_20120124_min.pdf
--> ctte_h_fed_st_1_20120123_min.pdf
--> ctte_h_fed_st_1_20120119_min.pdf
--> ctte_h_fed_st_1_20120118_min.pdf
--> ctte_h_fed_st_1_20120117_min.pdf
--> ctte_h_fed_st_1_20120111_min.pdf
--> ctte_h_fed_st_1_20120111_min.pdf
--> ctte_h_fed_st_1_20110505_min.pdf
--> ctte_h_fed_st_1_20110330_min.pdf
--> ctte_h_fed_st_1_20110322_min.pdf


/Users/zackushka/anaconda/envs/scr/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [128]:
b

,bill,session,text,url,filename
committee,,,,,
Committee on Federal and State Affairs,NaN,2011-2012 Legislative Sessions,Approved: _______3/26/12________ \n\nDate \nMI...,http://www.kslegislature.org/li_2012/b2011_12/...,ctte_h_fed_st_1_20120209_min.pdf
Committee on Federal and State Affairs,NaN,2011-2012 Legislative Sessions,Approved: _____3/26/12_________ \n\nDate \nMIN...,http://www.kslegislature.org/li_2012/b2011_12/...,ctte_h_fed_st_1_20120208_min.pdf
Committee on Federal and State Affairs,NaN,2011-2012 Legislative Sessions,Approved: ________2/24/12________ \n\nDate \nM...,http://www.kslegislature.org/li_2012/b2011_12/...,ctte_h_fed_st_1_20120207_min.pdf
Committee on Federal and State Affairs,NaN,2011-2012 Legislative Sessions,Approved: _______2/24/12_________ \n\nDate \nM...,http://www.kslegislature.org/li_2012/b2011_12/...,ctte_h_fed_st_1_20120206_min.pdf
Committee on Federal and State Affairs,NaN,2011-2012 Legislative Sessions,\n\nDate \nMINUTES OF THE HOUSE FEDERAL AND S...,http://www.kslegislature.org/li_2012/b2011_12/...,ctte_h_fed_st_1_20120202_min.pdf
Committee on Federal and State Affairs,NaN,2011-2012 Legislative Sessions,Date \nMINUTES OF THE HOUSE FEDERAL AND STATE ...,http://www.kslegislature.org/li_2012/b2011_12/...,ctte_h_fed_st_1_20120201_min.pdf
Committee on Federal and State Affairs,NaN,2011-2012 Legislative Sessions,Approved: ________2/07/12_______ \n\nDate \nMI...,http://www.kslegislature.org/li_2012/b2011_12/...,ctte_h_fed_st_1_20120131_min.pdf
Committee on Federal and State Affairs,NaN,2011-2012 Legislative Sessions,Approved: ________2/06/12________ \n\nDate \nM...,http://www.kslegislature.org/li_2012/b2011_12/...,ctte_h_fed_st_1_20120130_min.pdf
Committee on Federal and State Affairs,NaN,2011-2012 Legislative Sessions,Approved: ________2/06/12________ \n\nDate \nM...,http://www.kslegislature.org/li_2012/b2011_12/...,ctte_h_fed_st_1_20120126_min.pdf


In [125]:
b.to_csv('kansas_missed_20.csv')